In [1]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

In [2]:


# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 32.552794,
	"longitude": -117.047286,
	"hourly": ["temperature_2m", "wind_speed_10m", "wind_direction_10m","precipitation", "relative_humidity_2m"],
	"past_days": 30
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(1).ValuesAsNumpy()
hourly_wind_direction_10m = hourly.Variables(2).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(4).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(3).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["precipitation"] = hourly_precipitation

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

Coordinates 32.54777908325195°N -117.04148864746094°E
Elevation 16.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


TypeError: _generate_range() got an unexpected keyword argument 'inclusive'

In [3]:
hourly_dataframe.to_csv('../../../../data/weather/openmeteo/out/hourly.csv', index=False)


In [5]:
from foursquare.data_sdk import DataSDK, MediaType
import os
refresh_token=os.getenv("RC_FSQ_REFRESH_TOKEN")
#client = DataSDK(refresh_token=refresh_token)
client = DataSDK()

In [6]:
client.upload_dataframe(
            hourly_dataframe,
           dataset="42b2c74b-de4f-43e3-b93c-bb24a181f584",
            name='Weather',
            description="Weather From OpenMeteo via dataframe",)

Compressing file
Uploading: 100%|██████████| 15.3k/15.3k [00:00<00:00, 22.8kB/s]


Dataset(id=UUID('42b2c74b-de4f-43e3-b93c-bb24a181f584'), name='Weather', type=<DatasetType.MANAGED: 'managed'>, created_at=datetime.datetime(2025, 2, 13, 18, 16, 52, tzinfo=TzInfo(UTC)), updated_at=datetime.datetime(2025, 2, 14, 18, 33, 28, tzinfo=TzInfo(UTC)), description='Weather From OpenMeteo via dataframe', is_valid=True, data_connector=None, metadata=DatasetMetadata(media_type='text/csv', size=59069, source=None, tileset_data_url=None, tileset_metadata_url=None, image_url=None, metadata_url=None, data_status=None))

In [7]:
hourly_dataframe.set_index("date", inplace=True)


In [15]:
hourly_dataframe['wind_speed_10m']

date
2025-01-15 00:00:00+00:00     5.634891
2025-01-15 01:00:00+00:00     1.484318
2025-01-15 02:00:00+00:00    19.052811
2025-01-15 03:00:00+00:00    12.574260
2025-01-15 04:00:00+00:00    14.145756
                               ...    
2025-02-20 19:00:00+00:00     6.569383
2025-02-20 20:00:00+00:00     9.826088
2025-02-20 21:00:00+00:00    12.387348
2025-02-20 22:00:00+00:00    13.551501
2025-02-20 23:00:00+00:00    13.246826
Name: wind_speed_10m, Length: 888, dtype: float32

Add daily for max speed, and direction

Historica l data


In [9]:

cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 52.52,
	"longitude": 13.41,
	"start_date": "2024-01-01",
	"end_date": "2024-12-31",
	"hourly": ["temperature_2m", "wind_speed_10m", "wind_direction_10m", "wind_gusts_10m", "precipitation", "relative_humidity_2m"]
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(1).ValuesAsNumpy()
hourly_wind_direction_10m = hourly.Variables(2).ValuesAsNumpy()
hourly_wind_gusts_10m = hourly.Variables(3).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(4).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(5).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	closed = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
hourly_data["wind_gusts_10m"] = hourly_wind_gusts_10m
hourly_data["precipitation"] = hourly_precipitation
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m

hourly_dataframe = pd.DataFrame(data = hourly_data)
hourly_dataframe

Coordinates 52.5483283996582°N 13.407821655273438°E
Elevation 38.0 m asl
Timezone NoneNone
Timezone difference to GMT+0 0 s


,date,temperature_2m,wind_speed_10m,wind_direction_10m,wind_gusts_10m,precipitation,relative_humidity_2m
0,2024-01-01 00:00:00+00:00,5.0085,14.118243,199.359085,23.400000,0.0,86.903526
1,2024-01-01 01:00:00+00:00,5.4085,13.661038,198.435043,24.119999,0.0,85.421883
2,2024-01-01 02:00:00+00:00,5.0585,14.003029,197.969238,23.400000,0.0,85.081627
3,2024-01-01 03:00:00+00:00,4.6585,14.458382,198.886185,24.480000,0.0,85.643791
4,2024-01-01 04:00:00+00:00,4.4085,14.003029,197.969238,24.119999,0.0,86.535805
...,...,...,...,...,...,...,...
8779,2024-12-31 19:00:00+00:00,1.8585,16.928614,224.569260,34.919998,0.0,83.798660
8780,2024-12-31 20:00:00+00:00,1.5585,16.813387,217.606064,35.639999,0.0,84.068581
8781,2024-12-31 21:00:00+00:00,1.3085,18.108572,217.325592,37.799999,0.0,82.211578
8782,2024-12-31 22:00:00+00:00,1.1585,18.723461,217.971664,38.880001,0.0,81.890335
